In [19]:
from scipy.stats import stats
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
import os
from d import len_d
import pandas as pd
from pathlib import Path
import numpy as np

def MASE(A, F, y_train):
    return mean_absolute_scaled_error(A, F, y_train=y_train)


def SMAPE(a, f, _=None):
    return 1 / len(a) * np.sum(2 * np.abs(f - a) / (np.abs(a) + np.abs(f)) * 100)

In [20]:
autogluon = 'autogluon'
nbeats = 'nbeats'

ids_to_proceed = []
global_metrics = {}
targets= {}

In [22]:
metrics_f = {}
for i in os.listdir(autogluon):
    if i.endswith('csv'):
        idx = i.split('_')[0]
        ids_to_proceed.append(idx)
        df = pd.read_csv(Path(autogluon, i))
        #real_values = concated_df[concated_df['label'] == idx]['value'].values
        targets[idx] = df['value']
        metrics_f[idx] = SMAPE(df['value'], df['predict'])
global_metrics[autogluon] = metrics_f

metrics_f = {}
for i in os.listdir(nbeats):
    if i.endswith('csv'):
        idx = i.split('_')[0]
        ids_to_proceed.append(idx)
        df = pd.read_csv(Path(nbeats, i))
        #real_values = concated_df[concated_df['label'] == idx]['value'].values
        targets[idx] = df['value']
        metrics_f[idx] = SMAPE(df['value'], df['predict'])
global_metrics[nbeats] = metrics_f

In [25]:
df_full = pd.DataFrame()

#stats.ttest_rel(pre, post)

for k, v in global_metrics.items():
    df = pd.DataFrame(v, index=[k])
    df_full = pd.concat([df_full, df], axis=0)

df_full = df_full.dropna(axis=1)
for i in ['D', 'W',  'M', 'Q', 'Y', 'all']:
    quantile_df = pd.DataFrame()
    if i != 'all':
        filter_col = [col for col in df_full if col.startswith(i)]
        d__ = df_full[filter_col]
    else:
        d__ = df_full
    quantiles = [0.5]
    d__ = d__.dropna(axis=1)
    for q in quantiles:

        d_ = d__.quantile(q, axis=1).round(3)
        quantile_df = pd.concat([quantile_df, d_], axis=1)
        if np.isnan(d_[0]):
            continue

        #quantile_df.to_excel(f'{i}_quantiles.xlsx', index=True)
        print(i)
        print(f'{q} quantile')
        print('metric')
        for j, v in enumerate(d_.sort_values().index):
            print(j + 1, v, round(d_[v], 2))

    quantile_df.columns = quantiles

top_k = 1
wins_count = d__.apply(lambda x: x.nsmallest(top_k)).notnull().sum(axis=1).sort_values(ascending=False)

D
0.5 quantile
metric
1 nbeats 1.95
2 autogluon 2.37
W
0.5 quantile
metric
1 nbeats 4.38
2 autogluon 5.96
M
0.5 quantile
metric
1 autogluon 7.4
2 nbeats 7.63
Q
0.5 quantile
metric
1 autogluon 6.17
2 nbeats 8.19
Y
0.5 quantile
metric
1 autogluon 7.6
2 nbeats 12.65
all
0.5 quantile
metric
1 nbeats 4.64
2 autogluon 4.7


# Кол-во побед

In [26]:
wins_count

nbeats       435
autogluon    425
dtype: int64

# лейблы рядов

In [28]:
print(list(d__.columns.values))

['D1002', 'D1019', 'D1032', 'D1091', 'D1101', 'D1104', 'D1124', 'D1162', 'D1170', 'D1204', 'D1219', 'D1232', 'D1263', 'D1270', 'D1279', 'D1317', 'D1322', 'D1329', 'D133', 'D1368', 'D1378', 'D137', 'D1380', 'D1384', 'D1411', 'D1424', 'D1432', 'D1433', 'D1434', 'D1438', 'D1448', 'D1450', 'D1459', 'D145', 'D1464', 'D1475', 'D1519', 'D152', 'D1540', 'D1551', 'D1577', 'D1616', 'D1651', 'D1679', 'D1693', 'D1706', 'D1712', 'D1735', 'D1740', 'D1847', 'D1872', 'D1875', 'D1949', 'D1964', 'D1972', 'D1988', 'D2013', 'D2025', 'D2027', 'D2032', 'D2057', 'D2088', 'D2114', 'D2144', 'D2162', 'D2214', 'D2221', 'D2262', 'D2268', 'D2334', 'D2343', 'D237', 'D2393', 'D2484', 'D2486', 'D250', 'D2534', 'D2552', 'D2600', 'D261', 'D2648', 'D2670', 'D2698', 'D2717', 'D2729', 'D2797', 'D282', 'D2850', 'D2878', 'D2904', 'D292', 'D2979', 'D2991', 'D3006', 'D3012', 'D3023', 'D3029', 'D3043', 'D3046', 'D3050', 'D3055', 'D3067', 'D3082', 'D3083', 'D3085', 'D3094', 'D312', 'D3135', 'D3140', 'D3167', 'D3178', 'D319', 'D